# Model components

- SWAN config components to fully prescribe the INPUT command file
- Components are modular subclasses of `rompy.swan.components.base.BaseComponent`
- Arguments to each component are pydantic types or SWAN subcomponents
- The `model_type` attribute defines unique values to help parsing components from configs

In [2]:
import yaml
from pathlib import Path

from rompy.model import ModelRun
from rompy.swan.config import SwanConfigComponents

In [2]:
SwanConfigComponents?

Init signature:
SwanConfigComponents(
    *,
    model_type: Literal['swan'] = 'swan',
    template: str = '/source/csiro/rompy/rompy/templates/swan2',
    checkout: Optional[str] = 'main',
    project: rompy.swan.components.startup.PROJECT = None,
    set: rompy.swan.components.startup.SET = None,
    mode: rompy.swan.components.startup.MODE = None,
    coordinates: rompy.swan.components.startup.COORDINATES = None,
    cgrid: Union[rompy.swan.components.cgrid.REGULAR, rompy.swan.components.cgrid.CURVILINEAR, rompy.swan.components.cgrid.UNSTRUCTURED] = None,
    inpgrid: rompy.swan.components.inpgrid.INPGRIDS = None,
    boundary: Union[rompy.swan.components.boundary.BOUNDSPEC, rompy.swan.components.boundary.BOUNDNEST1, rompy.swan.components.boundary.BOUNDNEST2, rompy.swan.components.boundary.BOUNDNEST3] = None,
    initial: rompy.swan.components.boundary.INITIAL = None,
    physics: rompy.swan.components.physics.PHYSICS = None,
    **extra_data: Any,
) -> None
Docstring:      SWAN con

## Yaml config example

In [3]:
with open("swan-config-components.yml") as stream:
    config_dict = yaml.load(stream, Loader=yaml.Loader)
config_dict

{'project': {'model_type': 'project',
  'name': 'Test rompy setup',
  'nr': '0001'},
 'set': {'model_type': 'set',
  'level': 0.0,
  'depmin': 0.05,
  'maxerr': 1,
  'grav': 9.81,
  'rho': 1025.0,
  'cdcap': 0.0025,
  'direction_convention': 'nautical'},
 'mode': {'model_type': 'mode',
  'kind': 'nonstationary',
  'dim': 'twodimensional'},
 'coordinates': {'model_type': 'coordinates',
  'kind': {'model_type': 'spherical'}},
 'cgrid': {'model_type': 'regular',
  'spectrum': {'model_type': 'spectrum',
   'mdc': 36,
   'flow': 0.04,
   'fhigh': 0.4},
  'xlenc': 100.0,
  'ylenc': 100.0,
  'mxc': 10,
  'myc': 10},
 'inpgrid': {'inpgrids': [{'model_type': 'regular',
    'grid_type': 'bottom',
    'xpinp': 0.0,
    'ypinp': 0.0,
    'alpinp': 0.0,
    'mxinp': 10,
    'myinp': 10,
    'dxinp': 0.1,
    'dyinp': 0.1,
    'excval': -999.0,
    'readinp': {'model_type': 'readinp', 'fname1': 'bottom.txt'}},
   {'model_type': 'regular',
    'grid_type': 'wind',
    'xpinp': 0.0,
    'ypinp': 0.0,


In [4]:
swan_config = SwanConfigComponents(**config_dict)
swan_config

SwanConfigComponents(model_type='swan', template='/source/csiro/rompy/rompy/templates/swan2', checkout='main', project=PROJECT(model_type='project', name='Test rompy setup', nr='0001', title1=None, title2=None, title3=None), set=SET(model_type='set', level=0.0, nor=None, depmin=0.05, maxmes=None, maxerr=1, grav=9.81, rho=1025.0, cdcap=0.0025, inrhog=None, hsrerr=None, direction_convention='nautical', pwtail=None, froudmax=None, icewind=None), mode=MODE(model_type='mode', kind='nonstationary', dim='twodimensional'), coordinates=COORDINATES(model_type='coordinates', kind=SPHERICAL(model_type='spherical', projection='ccm'), reapeating=False), cgrid=REGULAR(model_type='regular', spectrum=SPECTRUM(model_type='spectrum', mdc=36, flow=0.04, fhigh=0.4, msc=None, dir1=None, dir2=None), xpc=0.0, ypc=0.0, alpc=0.0, xlenc=100.0, ylenc=100.0, mxc=10, myc=10), inpgrid=INPGRIDS(model_type='inpgrids', inpgrids=[REGULAR(model_type='regular', grid_type=<GridOptions.bottom: 'bottom'>, excval=-999.0, nons

### The render component method:

- Renders the component into the SWAN CMD string
- Uses the `key=value` convention so the order of parameters is not important
- Splits long commands to avoid exceeding Fortran maximum length

In [5]:
swan_config.project.render()

"PROJECT name='Test rompy setup' nr='0001'"

In [6]:
for component in config_dict.keys():
    print(getattr(swan_config, component).render() + "\n")

PROJECT name='Test rompy setup' nr='0001'

SET level=0.0 depmin=0.05 maxerr=1 grav=9.81 rho=1025.0 cdcap=0.0025 NAUTICAL

MODE NONSTATIONARY TWODIMENSIONAL

COORDINATES SPHERICAL CCM

CGRID REGULAR xpc=0.0 ypc=0.0 alpc=0.0 xlenc=100.0 ylenc=100.0 mxc=10 myc=10 CIRCLE mdc=36 flow=0.04 fhigh=0.4

INPGRID BOTTOM REGULAR xpinp=0.0 ypinp=0.0 alpinp=0.0 mxinp=10 myinp=10 dxinp=0.1 dyinp=0.1 EXCEPTION excval=-999.0
READINP BOTTOM fac=1.0 fname1='bottom.txt' idla=1 nhedf=0 nhedt=0 nhedvec=0 FREE

INPGRID WIND REGULAR xpinp=0.0 ypinp=0.0 alpinp=0.0 mxinp=10 myinp=10 dxinp=0.1 dyinp=0.1 EXCEPTION excval=-999.0 &
    NONSTATIONARY tbeginp=20230101.000000 deltinp=0.5 HR tendinp=20230201.000000
READINP WIND fac=1.0 fname1='wind.txt' idla=1 nhedf=0 nhedt=0 nhedvec=0 FREE

BOUND SHAPESPEC TMA gamma=3.3 d=12.0 PEAK DSPR POWER
BOUNDSPEC SIDE WEST CCW CONSTANT PAR hs=1.0 per=10.0 dir=0.0 dd=10.0

INITIAL HOTSTART SINGLE fname='hotfile.txt' FREE

GEN3 WESTHUYSEN DRAG WU
SSWELL ZIEGER
WCAPPING AB cds2=5e-

### Generating the INPUT file

In [7]:
# Helper functions to dump the contents of input and template

def dump_input(model):
    input_file = Path(model.output_dir) / model.run_id / "INPUT"
    print(input_file.read_text())

def dump_template(model):
    template_file = list(Path(model.config.template).glob("*"))[0] / "INPUT"
    print(template_file.read_text())

In [8]:
# Instantiate ModelRun with swan_config and the default template
model = ModelRun(
    run_id="tmp/test1",
    config=swan_config,
    output_dir="./",
)

# Generate the INPUT file
model.generate()

INFO:rompy.core.model:
INFO:rompy.core.model:-----------------------------------------------------
INFO:rompy.core.model:Model settings:
INFO:rompy.core.model:
period: 

	Start: 2020-02-21 04:00:00
	End: 2020-02-24 04:00:00
	Duration: 3 days, 0:00:00
	Interval: 0:15:00
	Include End: True


output_dir: 
./

config: 
model_type='swan' template='/source/csiro/rompy/rompy/templates/swan2' checkout='main' project=PROJECT(model_type='project', name='Test rompy setup', nr='0001', title1=None, title2=None, title3=None) set=SET(model_type='set', level=0.0, nor=None, depmin=0.05, maxmes=None, maxerr=1, grav=9.81, rho=1025.0, cdcap=0.0025, inrhog=None, hsrerr=None, direction_convention='nautical', pwtail=None, froudmax=None, icewind=None) mode=MODE(model_type='mode', kind='nonstationary', dim='twodimensional') coordinates=COORDINATES(model_type='coordinates', kind=SPHERICAL(model_type='spherical', projection='ccm'), reapeating=False) cgrid=REGULAR(model_type='regular', spectrum=SPECTRUM(model_typ

'/source/csiro/rompy/notebooks/components/tmp/test1'

In [9]:
# Examine the default template

dump_template(model)

!------------------------------------------------------------ Startup -------------------------------------------------------------

{% if config.project %}{{config.project.render()}}{% endif %}
{% if config.set != None %}{{config.set.render()}}{% endif %}
{% if config.mode != None %}{{config.mode.render()}}{% endif %}
{% if config.coordinates != None %}{{config.coordinates.render()}}{% endif %}


!------------------------------------------------------- Computational Grid -------------------------------------------------------

{% if config.cgrid %}{{config.cgrid.render()}}{% endif %}


!----------------------------------------------------------- Input Grids ----------------------------------------------------------

{% if config.inpgrid %}{{config.inpgrid.render()}}{% endif %}


!------------------------------------------------- Boundary and Initial conditions ------------------------------------------------

{% if config.boundary %}{{config.boundary.render()}}{% endif %}

{% if confi

In [10]:
# Examine the generated input

dump_input(model)

!------------------------------------------------------------ Startup -------------------------------------------------------------

PROJECT name='Test rompy setup' nr='0001'
SET level=0.0 depmin=0.05 maxerr=1 grav=9.81 rho=1025.0 cdcap=0.0025 NAUTICAL
MODE NONSTATIONARY TWODIMENSIONAL
COORDINATES SPHERICAL CCM


!------------------------------------------------------- Computational Grid -------------------------------------------------------

CGRID REGULAR xpc=0.0 ypc=0.0 alpc=0.0 xlenc=100.0 ylenc=100.0 mxc=10 myc=10 CIRCLE mdc=36 flow=0.04 fhigh=0.4


!----------------------------------------------------------- Input Grids ----------------------------------------------------------

INPGRID BOTTOM REGULAR xpinp=0.0 ypinp=0.0 alpinp=0.0 mxinp=10 myinp=10 dxinp=0.1 dyinp=0.1 EXCEPTION excval=-999.0
READINP BOTTOM fac=1.0 fname1='bottom.txt' idla=1 nhedf=0 nhedt=0 nhedvec=0 FREE

INPGRID WIND REGULAR xpinp=0.0 ypinp=0.0 alpinp=0.0 mxinp=10 myinp=10 dxinp=0.1 dyinp=0.1 EXCEPTION excval=-

## Mix up hardcoded and rendered commands

Use the same SWAN config object onto a modified template with the physics component hardcoded

In [12]:
swan_config.template = "./templates/swan-physics-predefined"
model = ModelRun(
    run_id="test2",
    config=swan_config,
    output_dir="./tmp",
)

In [14]:
model.generate()

INFO:rompy.core.model:
INFO:rompy.core.model:-----------------------------------------------------
INFO:rompy.core.model:Model settings:
INFO:rompy.core.model:
period: 

	Start: 2020-02-21 04:00:00
	End: 2020-02-24 04:00:00
	Duration: 3 days, 0:00:00
	Interval: 0:15:00
	Include End: True


output_dir: 
./tmp

config: 
model_type='swan' template='./templates/swan-physics-predefined' checkout='main' project=PROJECT(model_type='project', name='Test rompy setup', nr='0001', title1=None, title2=None, title3=None) set=SET(model_type='set', level=0.0, nor=None, depmin=0.05, maxmes=None, maxerr=1, grav=9.81, rho=1025.0, cdcap=0.0025, inrhog=None, hsrerr=None, direction_convention='nautical', pwtail=None, froudmax=None, icewind=None) mode=MODE(model_type='mode', kind='nonstationary', dim='twodimensional') coordinates=COORDINATES(model_type='coordinates', kind=SPHERICAL(model_type='spherical', projection='ccm'), reapeating=False) cgrid=REGULAR(model_type='regular', spectrum=SPECTRUM(model_type='

'/source/csiro/rompy/notebooks/components/tmp/test2'

In [15]:
# Examine the template

dump_template(model)

!------------------------------------------------------------ Startup -------------------------------------------------------------

{% if config.project %}{{config.project.render()}}{% endif %}
{% if config.set != None %}{{config.set.render()}}{% endif %}
{% if config.mode != None %}{{config.mode.render()}}{% endif %}
{% if config.coordinates != None %}{{config.coordinates.render()}}{% endif %}


!------------------------------------------------------- Computational Grid -------------------------------------------------------

{% if config.cgrid %}{{config.cgrid.render()}}{% endif %}


!----------------------------------------------------------- Input Grids ----------------------------------------------------------

{% if config.inpgrid %}{{config.inpgrid.render()}}{% endif %}


!------------------------------------------------- Boundary and Initial conditions ------------------------------------------------

{% if config.boundary %}{{config.boundary.render()}}{% endif %}

{% if confi

In [16]:
# And the generated INPUT

dump_input(model)

!------------------------------------------------------------ Startup -------------------------------------------------------------

PROJECT name='Test rompy setup' nr='0001'
SET level=0.0 depmin=0.05 maxerr=1 grav=9.81 rho=1025.0 cdcap=0.0025 NAUTICAL
MODE NONSTATIONARY TWODIMENSIONAL
COORDINATES SPHERICAL CCM


!------------------------------------------------------- Computational Grid -------------------------------------------------------

CGRID REGULAR xpc=0.0 ypc=0.0 alpc=0.0 xlenc=100.0 ylenc=100.0 mxc=10 myc=10 CIRCLE mdc=36 flow=0.04 fhigh=0.4


!----------------------------------------------------------- Input Grids ----------------------------------------------------------

INPGRID BOTTOM REGULAR xpinp=0.0 ypinp=0.0 alpinp=0.0 mxinp=10 myinp=10 dxinp=0.1 dyinp=0.1 EXCEPTION excval=-999.0
READINP BOTTOM fac=1.0 fname1='bottom.txt' idla=1 nhedf=0 nhedt=0 nhedvec=0 FREE

INPGRID WIND REGULAR xpinp=0.0 ypinp=0.0 alpinp=0.0 mxinp=10 myinp=10 dxinp=0.1 dyinp=0.1 EXCEPTION excval=-